In [ ]:
# default_exp mario

# End-to-End Pipeline

<br>

### Imports

In [2]:
#exports
import pandas as pd
import xarray as xr

from satip import eumetsat, reproj, io, gcp_helpers
from dagster import execute_pipeline, pipeline, solid, Field

import os
import dotenv

C:\Users\Ayrto\anaconda3\envs\satip_dev\lib\site-packages\google\auth\_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.21rows/s]


<br>

### User Inputs

In [3]:
data_dir = '../data/raw'
sorted_dir = '../data/sorted'
debug_fp = '../logs/EUMETSAT_download.txt'
env_vars_fp = '../.env'
metadata_db_fp = '../data/EUMETSAT_metadata.db'
new_grid_fp='../data/intermediate/new_grid_4km_TM.json'
new_coords_fp = '../data/intermediate/reproj_coords_TM_4km.csv'
zarr_bucket = 'solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/full_extent_TM_int16'

<br>

### Loading Environment Variables

In [11]:
#hide
_ = dotenv.load_dotenv(env_vars_fp)

In [4]:
#exports
user_key = os.environ.get('USER_KEY')
user_secret = os.environ.get('USER_SECRET')
slack_id = os.environ.get('SLACK_ID')
slack_webhook_url = os.environ.get('SLACK_WEBHOOK_URL')

<br>

### Dagster Pipeline

We're now going to combine these steps into a pipeline using `dagster`

In [5]:
#exports
@solid(
    config_schema = {
        'user_key': Field(str, default_value=user_key, is_required=False),
        'user_secret': Field(str, default_value=user_secret, is_required=False),
        'slack_webhook_url': Field(str, default_value=slack_webhook_url, is_required=False),
        'slack_id': Field(str, default_value=slack_id, is_required=False)
    }
)
def download_latest_eumetsat_files(context, data_dir: str, metadata_db_fp: str, debug_fp: str, table_id: str, project_id: str, start_date: str=''):
    if start_date == '':
        sql_query = f'select * from {table_id} where result_time = (select max(result_time) from {table_id})'
        start_date = gcp_helpers.query(sql_query, project_id)['result_time'].iloc[0]
        
    end_date = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')

    dm = eumetsat.DownloadManager(context.solid_config['user_key'], context.solid_config['user_secret'], data_dir, metadata_db_fp, debug_fp, slack_webhook_url=context.solid_config['slack_webhook_url'], slack_id=context.solid_config['slack_id'])
    df_new_metadata = dm.download_datasets(start_date, end_date)

    if df_new_metadata is None:
        df_new_metadata = pd.DataFrame(columns=['result_time', 'file_name'])
    else:
        df_new_metadata = df_new_metadata.iloc[1:] # the first entry is the last one we downloaded
        
    return df_new_metadata

@solid()
def df_metadata_to_dt_to_fp_map(_, df_new_metadata, data_dir: str) -> dict:
    """
    Here we'll then identify downloaded files in 
    the metadata dataframe and return a mapping
    between datetimes and filenames
    """
    
    datetime_to_filename = (df_new_metadata
                            .set_index('result_time')
                            ['file_name']
                            .drop_duplicates()
                            .to_dict()
                           )

    datetime_to_filepath = {
        datetime: f"{data_dir}/{filename}.nat" 
        for datetime, filename 
        in datetime_to_filename.items()
        if filename != {}
    }
    
    return datetime_to_filepath

@solid()
def reproject_datasets(_, datetime_to_filepath: dict, new_coords_fp: str, new_grid_fp: str):
    reprojector = reproj.Reprojector(new_coords_fp, new_grid_fp)

    reprojected_dss = [
        (reprojector
         .reproject(filepath, reproj_library='pyresample')
         .pipe(io.add_constant_coord_to_da, 'time', pd.to_datetime(datetime))
        )
        for datetime, filepath 
        in datetime_to_filepath.items()
    ]

    if len(reprojected_dss) > 0:
        ds_combined_reproj = xr.concat(reprojected_dss, 'time', coords='all', data_vars='all')
        return ds_combined_reproj
    else:
        return xr.Dataset()

@solid()
def compress_and_save_datasets(_, ds_combined_reproj, zarr_bucket: str, var_name: str='stacked_eumetsat_data'):
    # Handle case where no new data exists
    if len(ds_combined_reproj.dims) == 0:
        return
    
    # Compressing the datasets
    compressor = io.Compressor()

    var_name = var_name
    da_compressed = compressor.compress(ds_combined_reproj[var_name])

    # Saving to Zarr
    ds_compressed = io.save_da_to_zarr(da_compressed, zarr_bucket)
    
    return ds_compressed

@solid()
def save_metadata(_, df_new_metadata, table_id: str, project_id: str):
    gcp_helpers.write_metadata_to_gcp(df_new_metadata, table_id, project_id, append=True)

In [6]:
#exports
@pipeline
def download_latest_data_pipeline():  
    # Retrieving data, reprojecting, compressing, and saving to GCP
    df_new_metadata = download_latest_eumetsat_files()
    datetime_to_filepath = df_metadata_to_dt_to_fp_map(df_new_metadata)
    ds_combined_reproj = reproject_datasets(datetime_to_filepath)
    ds_combined_compressed = compress_and_save_datasets(ds_combined_reproj)
    save_metadata(df_new_metadata)

In [7]:
run_config = {
    'solids': {
        'download_latest_eumetsat_files': {
            'inputs': {
                'data_dir': "../data/raw",
                'metadata_db_fp': "../data/EUMETSAT_metadata.db",
                'debug_fp': "../logs/EUMETSAT_download.txt",
                'table_id': "eumetsat.metadata",
                'project_id': "solar-pv-nowcasting",
                'start_date': "2020-12-16 19:30"
            },
        },
        'df_metadata_to_dt_to_fp_map': {
            'inputs': {
                'data_dir': "../data/raw"
            }
        },
        'reproject_datasets': {
            'inputs': {
                'new_coords_fp': "../data/intermediate/reproj_coords_TM_4km.csv",
                'new_grid_fp': "../data/intermediate/new_grid_4km_TM.json"
            }
        },
        'compress_and_save_datasets': {
            'inputs': {
                'zarr_bucket': "solar-pv-nowcasting-data/satellite/EUMETSAT/SEVIRI_RSS/full_extent_TM_int16",
                'var_name': "stacked_eumetsat_data"
            }
        },
        'save_metadata': {
            'inputs': {
                'table_id': "eumetsat.metadata",
                'project_id': "solar-pv-nowcasting"
            },
        }
    }
}

execute_pipeline(download_latest_data_pipeline, run_config=run_config)

2020-12-16 20:00:18 - dagster - DEBUG - download_latest_data_pipeline - 843966a6-a689-430d-97e0-7c937aefc85e - 20040 - ENGINE_EVENT - Starting initialization of resources [asset_store].
2020-12-16 20:00:18 - dagster - DEBUG - download_latest_data_pipeline - 843966a6-a689-430d-97e0-7c937aefc85e - 20040 - ENGINE_EVENT - Finished initialization of resources [asset_store].
2020-12-16 20:00:18 - dagster - DEBUG - download_latest_data_pipeline - 843966a6-a689-430d-97e0-7c937aefc85e - 20040 - PIPELINE_START - Started execution of pipeline "download_latest_data_pipeline".
2020-12-16 20:00:18 - dagster - DEBUG - download_latest_data_pipeline - 843966a6-a689-430d-97e0-7c937aefc85e - 20040 - ENGINE_EVENT - Executing steps in process (pid: 20040)
2020-12-16 20:00:18 - dagster - DEBUG - download_latest_data_pipeline - 843966a6-a689-430d-97e0-7c937aefc85e - 20040 - download_latest_eumetsat_files.compute - STEP_START - Started execution of step "download_latest_eumetsat_files.compute".
2020-12-16 20:

AttributeError: 'DagsterLogManager' object has no attribute 'setLevel'

In [14]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_eumetsat.ipynb.
Converted 02_reproj.ipynb.
Converted 03_zarr.ipynb.
Converted 04_gcp.ipynb.
Converted 05_pipeline.ipynb.
Converted 103_loading.ipynb.
